In [1]:
import warnings; warnings.filterwarnings('ignore')
%matplotlib inline
import numpy as np
import oxyba as ox
from importlib import reload; reload(ox);

from sklearn.model_selection import train_test_split
from sklearn.datasets import load_boston

### Load Demo Dataset
At this point we will just load the demo dataset into a `tmp` list.

In [2]:
tmp = load_boston()

### Our Model
The model is an multivariate OLS regression (`oxyba.linreg_ols_lu`).
It is wrapped in `myfunc` what accept 1 variable `data` and returns the estimated coefficients.

In [3]:
def myfunc(data):
    import oxyba as ox;
    return ox.linreg_ols_lu( data[:,0], data[:,1:] );

The wrapper function fulfills the assumptions for the input argument `func` of the `oxyba.jackknife_loop` routine.

### First Try
The first model has an intercept, and two features as independent variables.
There is no further data processing.

In [4]:
varnames = ['intercept'] + list(tmp.feature_names[[5,12]])
y = tmp.target
X = np.c_[ np.ones(shape=(len(y),1)), tmp.data[:,[5,12]] ];

We will use just 10% of the full sample as training set

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.9, random_state=42)
N = len(y_train)
N

50

The Jackknife procedure makes a lot of sense for smaller dataset `N>=40`.
Thus, `N=50` is choosen as training set for demonstration purposes.

Let's compute the Jackknife with d=1 to 3.

In [6]:
for d in [1,2,3]:
    theta_subsample, theta_fullsample = ox.jackknife_loop(myfunc, np.c_[y_train, X_train], d=d);
    pvalues, tscores, theta_jack, se_jack, theta_biased = ox.jackknife_stats(theta_subsample, theta_fullsample, N=N, d=d)
    ox.jackknife_print(pvalues, tscores, theta_jack, se_jack, theta_biased, theta_fullsample, varnames, N, d)
    
    ins = ox.linreg_mse( y_train, X_train, theta_jack )
    out = ox.linreg_mse( y_test, X_test, theta_jack )
    print('{0:>30s}: {1:8.3f}'.format('In-sample MSE', ins))
    print('{0:>30s}: {1:8.3f}'.format('Out-of-sample MSE', out))


Delete-1 Jackknife, N=50
                                 intercept     RM        LSTAT   
                      p-Values:    0.62570    0.09172    0.09315 
                      t-Scores:   -0.49109    1.72239   -1.71460 
 Jackknife Standard Error (SE):   25.77396    3.75935    0.24130 
   Jackknife Estimates (theta):  -12.657      6.475     -0.414   
     Jackknife Biased Estimate:   -4.975      5.313     -0.461   
          Full Sample Estimate:   -5.128      5.336     -0.460   
                 In-sample MSE:   33.404
             Out-of-sample MSE:   32.116

Delete-2 Jackknife, N=50, C(N,d)=1225
                                 intercept     RM        LSTAT   
                      p-Values:    0.43110    0.04702    0.13620 
                      t-Scores:   -0.79446    2.04215   -1.51723 
 Jackknife Standard Error (SE):   25.67814    3.74443    0.24101 
   Jackknife Estimates (theta):  -20.400      7.647     -0.366   
     Jackknife Biased Estimate:   -4.817      5.289     -0.46

This does not look good. 
Why? Especially the intercept term is far away from being stable. 
The intercept's high p-value strongly indicates that the Jackknife Estimate (theta) is just chance.
You can even eyeball that the Jackknife Standard Error (SE) is relativly huge compared to the absolute Jackknife Estimate - It's almost 1-to-1. In case of the `RM` variables it is about 2-to-5 (pvalue 0.02). 

Conclusion: **We must reject this model**



### Second Try - Remove the Intercept Term
Again, the same, the same training data, the same independent variables except the intercept term

In [7]:
varnames = list(tmp.feature_names[[5,12]])
y = tmp.target
X = tmp.data[:,[5,12]];

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.9, random_state=42)
N = len(y_train)

for d in [1,2,3]:
    theta_subsample, theta_fullsample = ox.jackknife_loop(myfunc, np.c_[y_train, X_train], d=d);
    pvalues, tscores, theta_jack, se_jack, theta_biased = ox.jackknife_stats(theta_subsample, theta_fullsample, N=N, d=d)
    ox.jackknife_print(pvalues, tscores, theta_jack, se_jack, theta_biased, theta_fullsample, varnames, N, d)
    
    ins = ox.linreg_mse( y_train, X_train, theta_jack )
    out = ox.linreg_mse( y_test, X_test, theta_jack )
    print('{0:>30s}: {1:8.3f}'.format('In-sample MSE', ins))
    print('{0:>30s}: {1:8.3f}'.format('Out-of-sample MSE', out))


Delete-1 Jackknife, N=50
                                    RM        LSTAT   
                      p-Values:    0.00000    0.00055 
                      t-Scores:   15.87989   -3.70640 
 Jackknife Standard Error (SE):    0.28963    0.13362 
   Jackknife Estimates (theta):    4.599     -0.495   
     Jackknife Biased Estimate:    4.610     -0.506   
          Full Sample Estimate:    4.610     -0.506   
                 In-sample MSE:   33.197
             Out-of-sample MSE:   32.084

Delete-2 Jackknife, N=50, C(N,d)=1225
                                    RM        LSTAT   
                      p-Values:    0.00000    0.00072 
                      t-Scores:   15.83443   -3.62287 
 Jackknife Standard Error (SE):    0.28975    0.13372 
   Jackknife Estimates (theta):    4.588     -0.484   
     Jackknife Biased Estimate:    4.610     -0.506   
          Full Sample Estimate:    4.610     -0.506   
                 In-sample MSE:   33.228
             Out-of-sample MSE:   32.342



And the p-values indicate that the Jackknife Estimates (theta) are stable.
Compared to the 1st try, both `RM` and `LSTAT` coefficients improved a lot, i.e. lower Jackknife SE and as a result lower, more significant p-values.

As interesting side note, you will **not** detect parameter (in)stability from out-of-sample performance, model scores, etc. like MSE. The 1st and 2nd model perform about the same, and out-of-sample performance confirms the in-sample performance. 

Conclusion: **Model accepted**